#### Summer of Reproducibility - noWorkflow base experiment - Notebook 2

This Jupyter Notebook dedicated to walk you through noWorkflow applications in Data Science and Machine Learning. It is result of our work in the Summer of Reproducibility from OSPO UCSC 2023 with [noWorkflow](https://github.com/gems-uff/noworkflow).

This Notebook is an usecase based on the problem of Fraud Detection. We partially replicates the work "The effect of feature extraction and data sampling on credit card fraud detection". The interested reader is invited to consult the original work [here].(https://link.springer.com/article/10.1186/s40537-023-00684-w). 

For the sake of clarity, we splited this experiment in three steps into different Notebooks. 
1. Contemplates the steps from reading the dataset to Random Forest training, configuring one trial. 
2. Repeates all previous steps, but with changes in the experimental setup, like hyperparameters changed.
3. Repeates all precisou steps, changing the order and the modeling choice of the initial trial.
3. From previous results, noWorkflow is used to summarize and these trials

In [1]:
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

from noworkflow.now.tagging.var_tagging import backward_deps, \
    global_backward_deps, store_operations, resume_trials, trial_diff, \
    trial_intersection_diff, var_tag_plot, var_tag_values

#### Reading the dataset

In [2]:
df = pd.read_csv('dataset/creditcard.csv', encoding='utf-8')

### Feature engineering stage

Separate the features and target variable. First step in feature treatment.

In [3]:
X = df.drop('Class', axis=1)
y = df['Class']

#### Feature engineering: Apply PCA for feature extraction.

Here we define *pca_components* tag to keep the n_components argument in PCA

In [4]:
pca_components = now_variable('pca_components', 5)
pca = PCA(n_components=pca_components)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X)

Evaluation(id=33, checkpoint=29.209392186, code_component_id=870, activation_id=30, repr=5)


#### Feature engineering: Apply random undersampling over the extracted features

Another case of feature engineering operation with hyperparameter definition. Here is *random_state* value for RandomUnderSampler function


In [5]:
random_seed = now_variable('random_seed', 123456)
rus = RandomUnderSampler(random_state=random_seed)
X_resampled, y_resampled = rus.fit_resample(X_pca, y)

Evaluation(id=51, checkpoint=30.242671309, code_component_id=903, activation_id=48, repr=123456)


#### Feature engineering: Spliting dataset into train and test

Here we have two hyperparameters assignments: the proportion of the test_size and the random_state. 

In [6]:
test_dim = now_variable('test_dim', 0.3)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=test_dim, random_state=random_seed)

Evaluation(id=70, checkpoint=30.413706623, code_component_id=939, activation_id=67, repr=0.3)


#### Scoring: model training and transforming features into predictions
##### RandomForest

Instantiate and evaluate a Random Forest Classifier. Here we are tagging the model name in a model object 

In [7]:
rf = now_variable('model', RandomForestClassifier())
rf.fit(X_train, y_train)

Evaluation(id=89, checkpoint=30.522698816, code_component_id=973, activation_id=85, repr=RandomForestClassifier())


RandomForestClassifier()

#### Evaluating: evaluating the performance of models
##### RandomForest

Computing performance metrics. Two control variables are tagged here. *roc_rf* stores the ROC score classical metric in classification. On the other hand, *f1_rf* is the F1 score

In [8]:
y_pred_rf = rf.predict(X_test)

roc_rf = now_variable('roc_rf', roc_auc_score(y_test, y_pred_rf))
f1_rf = now_variable('f1_rf', f1_score(y_test, y_pred_rf))

print("Random Forest - ROC = %f, F1 = %f" % (roc_rf, f1_rf))

Evaluation(id=111, checkpoint=30.785813846, code_component_id=1010, activation_id=100, repr=0.9288127853881278)
Evaluation(id=120, checkpoint=30.788498385999997, code_component_id=1026, activation_id=100, repr=0.926829268292683)
Random Forest - ROC = 0.928813, F1 = 0.926829


### Experiment dependencies from roc_rf variable

In [9]:
dict_ops = backward_deps('roc_rf', False)
dict_ops

{26: ('y_test', 'complex data type'),
 25: ('RandomForestClassifier()', 'complex data type'),
 24: ("now_variable('model', RandomForestClassifier())", 'complex data type'),
 23: ('rf', 'complex data type'),
 22: ('X_resampled', 'complex data type'),
 21: ('RandomUnderSampler(random_state=random_seed)', 'complex data type'),
 20: ('rus', 'complex data type'),
 19: ("now_variable('pca_components', 5)", '5'),
 18: ('pca_components', '5'),
 17: ('PCA(n_components=pca_components)', 'PCA(n_components=5)'),
 16: ('pca', 'PCA(n_components=5)'),
 15: ('X', 'complex data type'),
 14: ('X_pca', 'complex data type'),
 13: ('df', 'complex data type'),
 12: ("df['Class']", 'complex data type'),
 11: ('y', 'complex data type'),
 10: ('y_resampled', 'complex data type'),
 9: ("now_variable('test_dim', 0.3)", '0.3'),
 8: ('test_dim', '0.3'),
 7: ("now_variable('random_seed', 123456)", '123456'),
 6: ('random_seed', '123456'),
 5: ('train_test_split(X_resampled, y_resampled, test_size=test_dim, random_s

### Experiment dependencies from roc_rf variable

Save the operations dictionary in a shelve object with this trial_id as a key.

Steps are:
1. calls store operations() to store the dict into a shelve object with this trial_id key
2. verify the list of stored trials available to comparision with resume_trials()

In [10]:
trial_id = __noworkflow__.trial_id
store_operations(trial_id, dict_ops)

Dictionary stored in shelve.


In [11]:
resume_trials()

['d17fbfb6-428c-466f-8734-44c3c399984d',
 'ad1f4984-7470-476f-aa31-6cb6ca348ea0']

### Next steps

In the next Notebook in this series, we will focus on comparing these two experiments with the noWorkflow 